# Invertible Syntax
##### (https://www.informatik.uni-marburg.de/~rendel/unparse/rendel10invertible.pdf)

We can write syntax descriptions that can be interpreted as both a Parser and a Printer. In other words, these are types that can interpreted polysemantically.

We employ partial isomorphisms to implement the invertible functions that are necessary for writing code that can be abstracted over Parser and Printer functionality.

In [2]:
data Iso a b = Iso (a -> Maybe b) (b -> Maybe a)

inverse :: Iso a b -> Iso b a
inverse (Iso f g) = Iso g f

apply :: Iso a b -> a -> Maybe b
apply (Iso f _) = f

unapply :: Iso a b -> b -> Maybe a
unapply = apply . inverse

This is needed because if we were to write a Functor instance that covers both Parser and Printer, the parser is doable because we first parse the value and then apply a function to it. Unfortunately this won't work for Printer because there is no use in printing a value and then applying a function to it - we would like to apply the function before printing (Printer is a contravariant functor).

Since both functions contained in an `Iso` are composable with a compatible `Iso` and we can also devise an identity `Iso`, it is fitting to write a `Category` instance.

In [16]:
import Prelude hiding ((.))
import Control.Category
import Control.Monad

instance Category Iso where
  g . f = Iso (apply f >=> apply g)
              (unapply g >=> unapply f)
  id = Iso Just Just

In order to define our syntax descriptors, we want to be able to use `Functor`, `Applicative`, and `Alternative` syntax which fully capture any type constructors. Before `Iso`s cannot be used with the general interfaces for these, we'll need specialized versions.

In [28]:
class IsoFunctor f where
  (<$$>) :: Iso a b -> f a -> f b
  
infixr 2 <$$>

We'll need to use a trick to provide the `Applicative` interface, otherwise the contravariance of Printer bites us once more (we end up with a function that takes a `b` but has no way of using it to produce the output). Whereas `<*>` supports curried function application, we will define an operator that uses uncurried function application, so instead of having a function of type `(Iso a b -> String) -> (a -> String) -> (b -> String)`, we'll have a function of type `(a -> String) -> (b -> String) -> (a, b) -> String`.

Printing with this function means printing the first component of the tuple with the first function and the second with the second function, while parsing means parsing an `a` value, then parsing a `b` value, and returning them as a tuple.

Using this, we can support the same `f <$> a <*> b` pattern except that in our version using right associative tupling and application of an uncurried function.

In [10]:
class ProductFunctor f where
  (<**>) :: f a -> f b -> f (a, b)
  
infixr 2 <**>

Supporting sum types is generally accomplished with the `Alternative` typeclass, which can be used for both Parsers and Printers but it has `Applicative` as a superclass constraint, so we must define it without that constraint.

In [11]:
class Alternative f where
  (<|>) :: f a -> f a -> f a
  empty :: f a

With these in hand, we can define the interface that provides the basic functionality for both Parsers and Printers.

In [41]:
class (IsoFunctor f, ProductFunctor f, Alternative f)
  => Syntax f where
  pureS :: Eq a => a -> f a
  -- ^ A Parser that produces some value without consuming any input or a Printer that checks if it's argument
  -- is equal to `a` and if so, discards it.
  token :: f Char
  -- ^ A Parser that simply hands you the next character from the input
  -- or a Printer that prints a single character.

Now we can define a function that parse/prints a list of parsable/printables:

In [42]:
-- Partial isomorphisms for constructors of the List type
nil :: Iso () [a]
nil = Iso (const $ Just [])
          (\xs -> case xs of
                    [] -> Just ()
                    _  -> Nothing
          )

cons :: Iso (a, [a]) [a]
cons = Iso (Just . uncurry (:))
           (\xs -> case xs of
                     [] -> Nothing
                     (x:xs) -> Just (x, xs)
           )
           
-- Function for parsing/printing a list of things
many :: Syntax f => f a -> f [a]
many p = cons <$$> p <**> many p
     <|> (nil <$$> pureS ())

Let's define simple `Parser` and `Printer` types with which we can implement this functionality.

In [59]:
newtype Parser a = Parser { runParser :: String -> [(a, String)] }
newtype Printer a = Printer { runPrinter :: a -> Maybe String }

parse :: Parser a -> String -> [a]
parse (Parser p) s = [x | (x, "") <- p s]

-- Parser instances

instance IsoFunctor Parser where
  Iso f _ <$$> (Parser p) = Parser $ \s -> [ (x', s')
                                           | (x, s') <- p s, Just x' <- [f x]
                                           ]
                                           
instance ProductFunctor Parser where
  Parser p1 <**> Parser p2 = Parser $ \s -> do
    (a, s') <- p1 s
    (b, s'') <- p2 s'
    [((a, b), s'')]
    
instance Alternative Parser where
  empty = Parser $ const []
  Parser p1 <|> Parser p2 = Parser $ (++) <$> p1 <*> p2
  
instance Syntax Parser where
  pureS a = Parser $ \s -> [(a, s)]
  token = Parser $ \s ->
    case s of
      c:s' -> [(c, s')]
      _ -> []
      
-- Printer instances

instance IsoFunctor Printer where
  Iso _ g <$$> Printer p = Printer $ p <=< g
  
instance ProductFunctor Printer where
  Printer p1 <**> Printer p2 = Printer $ \(a, b) -> (++) <$> p1 a <*> p2 b
  
instance Alternative Printer where
  empty = Printer $ const Nothing
  Printer p1 <|> Printer p2 = Printer $ \a ->
    p1 a `mplus` p2 a
      
instance Syntax Printer where
  pureS a = Printer $ \v ->
    if a == v then Just "" else Nothing
  token = Printer $ \x -> Just [x] 